In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from hydroeval import evaluator, nse, kge

2025-06-25 12:01:58.871329: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [122]:
durations = np.load("Results/Saved-from-run-models/durations-encdec-dyn-fusion.npy")
durations_for_preds = np.load("Results/Saved-from-run-models/durations-for-predictions-encdec-dyn-fusion.npy")
predictions_per_run = np.load("Results/Saved-from-run-models/predictions-per-run-encdec-dyn-fusion.npy")

In [17]:
n_stations=6
n_runs=10
def ensemble_predict(predictions): 
    predictions = np.array(predictions)
    avg_predictions = np.mean(predictions, axis=0)
    return avg_predictions
final_ensemble_predictions = []
for station_idx in range(n_stations):
    station_predictions_across_runs = [predictions_per_run[run][station_idx] for run in range(n_runs)]
    station_ensemble = ensemble_predict(station_predictions_across_runs)
    final_ensemble_predictions.append(station_ensemble)
np.save("Results/Saved-from-run-models/e-predictions-per-run-base-dyn1-fusion.npy", final_ensemble_predictions)

In [2]:
folder_with_data = 'Data/Final-data/'

river_flow = pd.read_csv(folder_with_data + '/river-flow.csv')
river_flow['dateTime'] = pd.to_datetime(river_flow['dateTime'], errors='raise')
river_flow['dateTime'] = river_flow['dateTime'].dt.date
river_flow = river_flow.groupby('dateTime').mean().reset_index()

climate_variables = pd.read_csv(folder_with_data + '/atmospheric-variables.csv')
climate_variables['dateTime'] = pd.to_datetime(climate_variables['dateTime'], errors='raise')
climate_variables['dateTime'] = climate_variables['dateTime'].dt.date
climate_variables = climate_variables.interpolate(method='linear', limit_direction='both', axis=0)

river_flow = river_flow.drop(columns={'Unnamed: 0'})
weather_data = pd.merge(climate_variables, river_flow, on=['dateTime'], how='inner')

weather_data = weather_data.iloc[:,[0,3,1,4,5,6,7,8,9,10,11,12,13]]
time_steps = 30 
horizon = 7 


/var/folders/th/mt68jv4d1yg0fk5k079pdmdr0000gn/T/ipykernel_2319/167017545.py:12: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  climate_variables = climate_variables.interpolate(method='linear', limit_direction='both', axis=0)


In [3]:
time_steps = 30 
horizon = 7 

numerical_columns = weather_data.select_dtypes(include=['float64']).columns
datetime_columns = weather_data.select_dtypes(include=['datetime64']).columns

scaler = MinMaxScaler(feature_range=(0,1))
weather_data[numerical_columns]= pd.DataFrame(scaler.fit_transform(weather_data[numerical_columns]), columns = numerical_columns)

In [64]:
Y =  weather_data.iloc[:, 8:] 
Y1 =  weather_data.iloc[:,8] 
Y2 =  weather_data.iloc[:,9] 
Y3 =  weather_data.iloc[:,10] 
Y4 =  weather_data.iloc[:,11] 
Y5 =  weather_data.iloc[:,12] 
Y6 =  weather_data.iloc[:,13] 
X1 = weather_data.iloc[:, [1]]  
X2 = weather_data.iloc[:, [2]] 
X3 = weather_data.iloc[:, [3]] 
X4 = weather_data.iloc[:, [4]] 
X5 = weather_data.iloc[:, [5]] 
X6 = weather_data.iloc[:, [6]] 
X7 = weather_data.iloc[:, [7]] 
time = weather_data.iloc[:, [0]] 

def create_sequences(X1, X2, X3, X4, X5, X6, 
                     X7,
                     Y, time, time_steps, horizon):
    
    X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, X7_seq = [], [], [], [], [], [], []
    
    Y_past_seq = []
    
    Y_seq = []
    
    time_seq = []

    for i in range(len(Y) - time_steps - horizon + 1):
        X1_seq.append(X1[i:i+time_steps])
        X2_seq.append(X2[i:i+time_steps])
        X3_seq.append(X3[i:i+time_steps])
        X4_seq.append(X4[i:i+time_steps])
        X5_seq.append(X5[i:i+time_steps])
        X6_seq.append(X6[i:i+time_steps])
        X7_seq.append(X7[i:i+time_steps])

        Y_past_seq.append(Y.iloc[i:i+time_steps]) 

        Y_seq.append(Y.iloc[i+time_steps:i+time_steps+horizon])  

        time_seq.append(time[i:i+time_steps])


    X1_seq = np.array(X1_seq)
    X2_seq = np.array(X2_seq)
    X3_seq = np.array(X3_seq)
    X4_seq = np.array(X4_seq)
    X5_seq = np.array(X5_seq)
    X6_seq = np.array(X6_seq)
    X7_seq = np.array(X7_seq)
    
    Y_past_seq = np.array(Y_past_seq)
    
    Y_seq = np.array(Y_seq)
    
    time_seq = np.array(time_seq)

    return (X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq,
            X7_seq,
            Y_past_seq,
            Y_seq, time_seq)

# Assuming time_steps and horizon are defined
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, X7_seq, Y1_past_seq, Y1_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, X7, Y1, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, X7_seq, Y2_past_seq, Y2_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, X7, Y2, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, X7_seq, Y3_past_seq, Y3_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, X7, Y3, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, X7_seq, Y4_past_seq, Y4_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, X7, Y4, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq,  X7_seq, Y5_past_seq, Y5_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, X7, Y5, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq,  X7_seq, Y6_past_seq, Y6_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, X7, Y6, time, time_steps, horizon
)

modality_1 = Y1_past_seq
modality_2 = Y2_past_seq
modality_3 = Y3_past_seq
modality_4 = Y4_past_seq
modality_5 = Y5_past_seq
modality_6 = Y6_past_seq

# Split into training and testing sets
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, X7_train, X7_test, y1_past_train, y1_past_test,  y1_train, y1_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq,  X7_seq, Y1_past_seq, Y1_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, X7_train, X7_test, y2_past_train, y2_past_test,  y2_train, y2_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq,  X7_seq, Y2_past_seq, Y2_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, X7_train, X7_test, y3_past_train, y3_past_test,  y3_train, y3_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq,  X7_seq, Y3_past_seq, Y3_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, X7_train, X7_test, y4_past_train, y4_past_test,  y4_train, y4_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq,  X7_seq, Y4_past_seq, Y4_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, X7_train, X7_test, y5_past_train, y5_past_test,  y5_train, y5_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq,  X7_seq, Y5_past_seq, Y5_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, X7_train, X7_test, y6_past_train, y6_past_test,  y6_train, y6_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq,  X7_seq, Y6_past_seq, Y6_seq, test_size=0.2, shuffle=False)

train_time, test_time = train_test_split(time_seq, test_size=0.2, shuffle=False)


In [4]:
Y =  weather_data.iloc[:, 7:]  
Y1 =  weather_data.iloc[:,7] 
Y2 =  weather_data.iloc[:,8] 
Y3 =  weather_data.iloc[:,9] 
Y4 =  weather_data.iloc[:,10] 
Y5 =  weather_data.iloc[:,11] 
Y6 =  weather_data.iloc[:,12] 
X1 = weather_data.iloc[:, [1]]  
X2 = weather_data.iloc[:, [2]] 
X3 = weather_data.iloc[:, [3]] 
X4 = weather_data.iloc[:, [4]] 
X5 = weather_data.iloc[:, [5]] 
X6 = weather_data.iloc[:, [6]] 
time = weather_data.iloc[:, [0]] 

def create_sequences(X1, X2, X3, X4, X5, X6, 
                     Y, time, time_steps, horizon):
    
    X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq = [], [], [], [], [], []
    
    Y_past_seq = []
    
    Y_seq = []
    
    time_seq = []

    for i in range(len(Y) - time_steps - horizon + 1):
        X1_seq.append(X1[i:i+time_steps])
        X2_seq.append(X2[i:i+time_steps])
        X3_seq.append(X3[i:i+time_steps])
        X4_seq.append(X4[i:i+time_steps])
        X5_seq.append(X5[i:i+time_steps])
        X6_seq.append(X6[i:i+time_steps])
        
        Y_past_seq.append(Y.iloc[i:i+time_steps]) 

        Y_seq.append(Y.iloc[i+time_steps:i+time_steps+horizon])  

        time_seq.append(time[i:i+time_steps])

    X1_seq = np.array(X1_seq)
    X2_seq = np.array(X2_seq)
    X3_seq = np.array(X3_seq)
    X4_seq = np.array(X4_seq)
    X5_seq = np.array(X5_seq)
    X6_seq = np.array(X6_seq)
    
    Y_past_seq = np.array(Y_past_seq)
    
    Y_seq = np.array(Y_seq)
    
    time_seq = np.array(time_seq)

    return (X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, 
            Y_past_seq,
            Y_seq, time_seq)

X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y1_past_seq, Y1_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, Y1, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y2_past_seq, Y2_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, Y2, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y3_past_seq, Y3_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, Y3, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y4_past_seq, Y4_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, Y4, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y5_past_seq, Y5_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, Y5, time, time_steps, horizon
)
X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y6_past_seq, Y6_seq, time_seq = create_sequences(
    X1, X2, X3, X4, X5, X6, Y6, time, time_steps, horizon
)

modality_1 = Y1_past_seq
modality_2 = Y2_past_seq
modality_3 = Y3_past_seq
modality_4 = Y4_past_seq
modality_5 = Y5_past_seq
modality_6 = Y6_past_seq

# Split into training and testing sets
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, y1_past_train, y1_past_test,  y1_train, y1_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y1_past_seq, Y1_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, y2_past_train, y2_past_test,  y2_train, y2_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y2_past_seq, Y2_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, y3_past_train, y3_past_test,  y3_train, y3_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y3_past_seq, Y3_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, y4_past_train, y4_past_test,  y4_train, y4_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y4_past_seq, Y4_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, y5_past_train, y5_past_test,  y5_train, y5_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y5_past_seq, Y5_seq, test_size=0.2, shuffle=False)
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, X6_train, X6_test, y6_past_train, y6_past_test,  y6_train, y6_test  = train_test_split(X1_seq, X2_seq, X3_seq, X4_seq, X5_seq, X6_seq, Y6_past_seq, Y6_seq, test_size=0.2, shuffle=False)

train_time, test_time = train_test_split(time_seq, test_size=0.2, shuffle=False)


In [5]:
ys_test = []
scaler_modified = MinMaxScaler(feature_range=(0, 1))
scaler_modified.feature_range = scaler.feature_range
for j, y_test in enumerate([y1_test, y2_test, y3_test, y4_test, y5_test, y6_test]): 
    scaler_modified.min_ =  scaler.min_[6+j]
    scaler_modified.scale_ =  scaler.scale_[6+j]
    scaler_modified.data_min_ =  scaler.data_min_[6+j]
    scaler_modified.data_max_ =  scaler.data_max_[6+j]
    scaler_modified.data_range_ =  scaler.data_range_[6+j]

    y_test = scaler_modified.inverse_transform(y_test)
    ys_test.append(y_test)

In [6]:
predictions_encdec_un = np.load('e-predictions-per-run-encdec-dyn-fusion.npy')
predictions_encdec_dyn_un = np.load('e-predictions-per-run-encdec-dyn1-fusion.npy')

In [107]:
predictions_encdec_un = np.load('predictions-per-run-encdec-dyn-fusion.npy')[8]
predictions_encdec_dyn_un = np.load('predictions-per-run-encdec-dyn1-fusion.npy')[8]

In [9]:

predictions_per_model = []
scaler_modified = MinMaxScaler(feature_range=(0, 1))
scaler_modified.feature_range = scaler.feature_range
for model in [predictions_encdec_un,predictions_encdec_dyn_un]:
    predictions = []
    for j, model_per_station in enumerate(model): 
        scaler_modified.min_ =  scaler.min_[6+j]
        scaler_modified.scale_ =  scaler.scale_[6+j]
        scaler_modified.data_min_ =  scaler.data_min_[6+j]
        scaler_modified.data_max_ =  scaler.data_max_[6+j]
        scaler_modified.data_range_ =  scaler.data_range_[6+j]
        model_per_station = model_per_station.reshape(-1,1)
        prediction = scaler_modified.inverse_transform(model_per_station)
        predictions.append(prediction)

    predictions_per_model.append(predictions)

In [10]:
y_test_combined = np.stack(ys_test, axis=1).transpose(0,2,1)


In [12]:
y_test_combined = y_test_combined.reshape(-1,6)

In [13]:
actuals = y_test_combined 

In [15]:
predictions_per_run =[predictions_encdec_un,predictions_encdec_dyn_un]
predictions = []
for prediction in predictions_per_run:
        predictions_per_run2 = []
        for j in range(prediction.shape[0]): 
            data_slice = prediction[j, :, :, 0]
            scaler_modified.min_ = scaler.min_[6+j]
            scaler_modified.scale_ = scaler.scale_[6+j]
            scaler_modified.data_min_ = scaler.data_min_[6+j]
            scaler_modified.data_max_ = scaler.data_max_[6+j]
            scaler_modified.data_range_ = scaler.data_range_[6+j]

            data_inverse_transformed = scaler_modified.inverse_transform(data_slice)
        
            predictions_per_run2.append(data_inverse_transformed)
        predictions.append(predictions_per_run2)

In [16]:
def calculate_mase(actual, predicted, m):
    n = len(actual)
    naive_error = np.abs(actual[m:] - actual[:-m])  
    forecast_error = np.abs(actual[m:] - predicted[m:])
    scaled_error = np.mean(forecast_error) / np.mean(naive_error)
    return scaled_error

def nash_sutcliffe_efficiency(observed, predicted):
    observed = np.array(observed)
    predicted = np.array(predicted)

    numerator = np.sum((observed - predicted) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)

    return 1 - (numerator / denominator)

In [17]:
num_stations = 6
error_metrics = []
mae_high_per_model = []
mae_per_model = []
nse_per_model = []
mase_per_model = []

models = ["LSTM-EncDec-Att-Fus",
          "LSTM-EncDec-Op-Lev-Fus"]

for model_name, model in zip(models,predictions_per_model):
    model =  np.array(model)
    model_predictions = model.transpose(1,2,0).reshape(-1,6)

    df = pd.DataFrame({'timestamp': range(0,len(actuals.flatten())),'Flow': actuals.flatten()})

    threshold_high = df['Flow'].quantile(0.95)  
    threshold_low = df['Flow'].quantile(0.05)   

    extreme_highs = df[df['Flow'] >= threshold_high]
    extreme_lows = df[df['Flow'] <= threshold_low]

    df2 = pd.DataFrame({'timestamp': range(0,len(model_predictions.flatten()+1)),'Flow': model_predictions.flatten() })

    threshold_high2 = df['Flow'].quantile(0.95)  
    threshold_low2 = df['Flow'].quantile(0.05)   

    extreme_highs2 = df2[df['Flow'] >= threshold_high2]
    extreme_lows2 = df2[df['Flow'] <= threshold_low2]

    mae = round(mean_absolute_error(np.array(df['Flow']), np.array(df2['Flow'])),2)
    nse = round(nash_sutcliffe_efficiency(np.array(df['Flow']), np.array(df2['Flow'])),2)
    mase = round(calculate_mase(np.array(df['Flow']), np.array(df2['Flow']),20),2)
    mae_high = round(mean_absolute_error(extreme_highs['Flow'], extreme_highs2['Flow']),2)
    
    mae_high_per_model.append(mae_high)
    mase_per_model.append(mase)
    mae_per_model.append(mae)
    nse_per_model.append(nse)

    error_metrics_df = pd.DataFrame({
        'Model' : [model_name],
        'MAE' : [mae],
        'NSE' : [nse],
        'MASE': [mase],
        'MAE_HIGH': [mae_high]
    })

    error_metrics.append(error_metrics_df)

    
error_metrics = pd.concat(error_metrics,axis=0).reset_index(drop=True)
print(error_metrics)

                                Model    MAE   NSE  MASE  MAE_HIGH  \
0   LSTM-EncDec-Intra-Inter-Attention  27.73  0.75  0.53    109.59   
1             LSTM-EncDec-Dynamic-Fus  26.29  0.76  0.50    105.62   
2  1DCNN-BiLSTM-Intra-Inter-Attention  28.22  0.75  0.54    111.07   
3            1DCNN-BiLSTM-Dynamic-Fus  26.32  0.76  0.50    106.85   

   RQE_HIGH_90  RQE_HIGH_95  RQE_HIGH_99  
0         0.13         0.15         0.18  
1         0.14         0.14         0.17  
2         0.15         0.16         0.18  
3         0.15         0.14         0.17  


In [28]:
error_metrics.to_csv('error_metrics_un.csv')

In [18]:
# Predictions per station
num_stations = 6
error_metrics = []

models = [
    "LSTM-EncDec-Att-Fus",
    "LSTM-EncDec-Op-Lev-Fus"
]

for model_name, model in zip(models,predictions_per_model):
  mae_high_per_station = []
  mae_per_station = []
  nse_per_station = []
  mase_per_station = []
  extreme_lows_per_station = []
  extreme_highs_per_station = []
  model =  np.array(model)
  model = model.transpose(1,2,0).reshape(-1,6)

  for i,name in zip(range(num_stations),weather_data.columns[7:]):

      df = pd.DataFrame({'timestamp': range(0,len(actuals[:,i]+1)),'Flow': actuals[:,i]})

      threshold_high = df['Flow'].quantile(0.95)  

      extreme_highs = df[df['Flow'] >= threshold_high]

      df2 = pd.DataFrame({'timestamp': range(0,len(actuals[:,i]+1)),'Flow': model[:,i].flatten()})

      threshold_high2 = df['Flow'].quantile(0.95)  

      extreme_highs2 = df2[df['Flow'] >= threshold_high2]

      extreme_lows_per_station.append(extreme_lows)
      extreme_highs_per_station.append(extreme_highs)

      mae_high = round(mean_absolute_error(extreme_highs['Flow'], extreme_highs2['Flow']),2)
      mae_low = round(mean_absolute_error(extreme_lows['Flow'], extreme_lows2['Flow']),2)
      mae_high_per_station.append(mae_high)
      mae = round(mean_absolute_error(np.array(df['Flow']), np.array(df2['Flow'])),2)
      mae_per_station.append(mae)

      nse = round(nash_sutcliffe_efficiency(np.array(df['Flow']), np.array(df2['Flow'])),2)
      nse_per_station.append(nse)
      mase = round(calculate_mase(np.array(df['Flow']), np.array(df2['Flow']),20),2)
      mase_per_station.append(mase)

      actual_quantiles_90 = df['Flow'].quantile(0.90)
      predicted_quantiles_90 = df2['Flow'].quantile(0.90)
      rqe_90 = round(np.abs(predicted_quantiles_90 - actual_quantiles_90) / actual_quantiles_90,2)
      predicted_quantiles_95 = df2['Flow'].quantile(0.95)
      rqe_95 = round(np.abs(predicted_quantiles_95 - actual_quantiles_95) / actual_quantiles_95,2)
      actual_quantiles_99 = df['Flow'].quantile(0.99)
      predicted_quantiles_99 = df2['Flow'].quantile(0.99)
      rqe_99 = round(np.abs(predicted_quantiles_99 - actual_quantiles_99) / actual_quantiles_99,2)
      actual_prob = df['Flow'] / np.sum(df['Flow'])
      predicted_prob = df2['Flow'] / np.sum(df2['Flow'])
      predicted_prob = np.clip(predicted_prob, 1e-10, None)  

      quantile_95 = np.percentile(df2['Flow'], 95)
      lower_bound = np.percentile(df2['Flow'], 90)  
      upper_bound = quantile_95  
      quantile_99 =   np.percentile(df2['Flow'], 99)
      lower_bound_2 = np.percentile(df2['Flow'], 95) 
      upper_bound_2 = quantile_99 
      lower_bound_3 = np.percentile(df2['Flow'], 10) 
      upper_bound_3 = lower_bound
      # within_interval = (df2['Flow'] >= lower_bound) & (df2['Flow'] <= upper_bound)
      within_interval_3 = (df['Flow'] >= lower_bound_3) & (df['Flow'] <= upper_bound_3)
      within_interval_2 = (df['Flow'] >= lower_bound_2) & (df['Flow'] <= upper_bound_2)
      within_interval = (df['Flow'] >= lower_bound) & (df['Flow'] <= upper_bound)
      pci_3 = round(np.mean(within_interval_3) * 100,2)
      pci_2 = round(np.mean(within_interval_2) * 100,2)
      pci = round(np.mean(within_interval) * 100,2)

      overlap = np.intersect1d(extreme_highs, extreme_highs2)
      overlap_percentage = round(len(overlap) / len(extreme_highs) * 100,2)

  error_metrics_df = pd.DataFrame({
      'Model' : model_name,
      'Station': [f'{name}' for name in weather_data.columns[7:]],
      'MAE': mae_per_station,
      'NSE' : nse_per_station,
      'MASE': mase_per_station,
      'MAE_HIGH': mae_high_per_station

  })

  error_metrics.append(error_metrics_df)

error_metrics = pd.concat(error_metrics,axis=0).reset_index(drop=True)
print(error_metrics)
  


                      Model Station    MAE   NSE  MASE  MAE_HIGH
0       LSTM-EncDec-Att-Fus   54001  23.87  0.71  1.12     95.92
1       LSTM-EncDec-Att-Fus   54005  18.29  0.66  1.13     67.03
2       LSTM-EncDec-Att-Fus   54032  32.95  0.78  1.15     96.23
3       LSTM-EncDec-Att-Fus   54057  39.58  0.77  1.15    130.81
4       LSTM-EncDec-Att-Fus   54095  25.79  0.65  1.08    124.13
5       LSTM-EncDec-Att-Fus   55002  25.88  0.48  1.03    142.14
6    LSTM-EncDec-Op-Lev-Fus   54001  22.82  0.72  1.07     93.27
7    LSTM-EncDec-Op-Lev-Fus   54005  17.28  0.68  1.07     63.85
8    LSTM-EncDec-Op-Lev-Fus   54032  31.50  0.79  1.10     96.86
9    LSTM-EncDec-Op-Lev-Fus   54057  37.59  0.78  1.09    121.66
10   LSTM-EncDec-Op-Lev-Fus   54095  23.65  0.67  0.99    120.78
11   LSTM-EncDec-Op-Lev-Fus   55002  24.92  0.50  0.99    137.73
12     1DCNN-BiLSTM-Att-Fus   54001  24.32  0.70  1.14     98.42
13     1DCNN-BiLSTM-Att-Fus   54005  18.77  0.66  1.16     66.15
14     1DCNN-BiLSTM-Att-F

In [29]:
np.save('predictions_per_model_un.npy',predictions_per_model)